In [1]:
## Import Libraries
from sklearn.neural_network import MLPClassifier
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import pickle
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import joblib
import pickle
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from tensorflow.keras import Sequential,layers, losses, optimizers
from tensorflow.keras.layers import Dense
import tensorflow as tf

In [2]:
## Loading CICIDS2017 training data
df_train = pd.read_csv("all_group_train_normalized.csv")

In [81]:
df_train['Label'].value_counts()

Benign          53432
DoS              8906
Web Attack       8906
Infiltration     8905
Port Scan        8905
DDoS             8905
Brute Force      8905
Name: Label, dtype: int64

In [77]:
## Loading CICIDS2017 testing data
df_test = pd.read_csv("all_group_test_normalized.csv")

In [79]:
## Subsetting different attack packets
infiltration_attack = df_test[df_test['Label'] == 'Infiltration'].drop(['Label_binary','Label'], axis=1).iloc[:,0:200]
print(infiltration_attack.shape)
portscan_attack = df_test[df_test['Label'] == 'Port Scan'].drop(['Label_binary','Label'], axis=1).iloc[:,0:200]
print(portscan_attack.shape)
ddos_attack = df_test[df_test['Label'] == 'DDoS'].drop(['Label_binary','Label'], axis=1).iloc[:,0:200]
print(ddos_attack.shape)
bruteforce_attack = df_test[df_test['Label'] == 'Brute Force'].drop(['Label_binary','Label'], axis=1).iloc[:,0:200]
print(bruteforce_attack.shape)
dos_attack = df_test[df_test['Label'] == 'DoS'].drop(['Label_binary','Label'], axis=1).iloc[:,0:200]
print(dos_attack.shape)
web_attack = df_test[df_test['Label'] == 'Web Attack'].drop(['Label_binary','Label'], axis=1).iloc[:,0:200]
print(web_attack.shape)

(3817, 200)
(3817, 200)
(3817, 200)
(3817, 200)
(3816, 200)
(3816, 200)


In [101]:
## Loading CICIDS-2018 data
df_test_2018 = pd.read_csv("all_group_train_normalized_2018.csv")

In [144]:
df_test_2018['Label'].value_counts()

Benign          145579
DoS              29116
Web Attack       29116
Infiltration     29116
Brute Force      29116
ddos             29115
Name: Label, dtype: int64

In [148]:
infiltration_attack = df_test_2018[df_test_2018['Label'] == 'Infiltration'].drop(['Label_binary','Label'], axis=1).iloc[:,0:1525]
print(infiltration_attack.shape)
portscan_attack = df_test_2018[df_test_2018['Label'] == 'Port Scan'].drop(['Label_binary','Label'], axis=1).iloc[:,0:1525]
print(portscan_attack.shape)
ddos_attack = df_test_2018[df_test_2018['Label'] == 'ddos'].drop(['Label_binary','Label'], axis=1).iloc[:,0:1525]
print(ddos_attack.shape)
bruteforce_attack = df_test_2018[df_test_2018['Label'] == 'Brute Force'].drop(['Label_binary','Label'], axis=1).iloc[:,0:1525]
print(bruteforce_attack.shape)
dos_attack = df_test_2018[df_test_2018['Label'] == 'DoS'].drop(['Label_binary','Label'], axis=1).iloc[:,0:1525]
print(dos_attack.shape)
web_attack = df_test_2018[df_test_2018['Label'] == 'Web Attack'].drop(['Label_binary','Label'], axis=1).iloc[:,0:1525]
print(web_attack.shape)

(29116, 1525)
(0, 1525)
(29115, 1525)
(29116, 1525)
(29116, 1525)
(29116, 1525)


In [103]:
X_2018 = df_test_2018.iloc[:, :1525]

In [5]:
X_train = df_train.iloc[:,:1525]
Y_train = df_train.iloc[:,1526:1527]

In [7]:
Y_train.loc[Y_train['Label_binary'] == 'Benign', 'Label_binary'] = 0

In [8]:
Y_train.loc[Y_train['Label_binary'] == 'Malicious', 'Label_binary'] = 1

In [9]:
X_train_ = X_train.values.astype('float32')

In [10]:
Y_train_ = Y_train.values.astype('float32')

In [57]:
## The DNN model training
tf.random.set_seed(2)
input_dims = 1525
model = Sequential()
model.add(Dense(input_dims, input_shape = (input_dims, )))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [58]:
## Compilation and training of the model
model.compile(optimizer='adam', loss= 'binary_crossentropy', metrics=['accuracy'])
model.fit(X_train_ , Y_train_, batch_size=1024, epochs=50, shuffle=True)

Epoch 1/50
105/105 [==============================] - 4s 31ms/step - loss: 0.1411 - accuracy: 0.9545
Epoch 2/50
105/105 [==============================] - 3s 32ms/step - loss: 0.0769 - accuracy: 0.9762
Epoch 3/50
105/105 [==============================] - 4s 36ms/step - loss: 0.0574 - accuracy: 0.9838
Epoch 4/50
105/105 [==============================] - 3s 32ms/step - loss: 0.0412 - accuracy: 0.9888
Epoch 5/50
105/105 [==============================] - 3s 32ms/step - loss: 0.0375 - accuracy: 0.9890
Epoch 6/50
105/105 [==============================] - 3s 33ms/step - loss: 0.0273 - accuracy: 0.9919
Epoch 7/50
105/105 [==============================] - 4s 34ms/step - loss: 0.0239 - accuracy: 0.9928
Epoch 8/50
105/105 [==============================] - 4s 36ms/step - loss: 0.0235 - accuracy: 0.9929
Epoch 9/50
105/105 [==============================] - 3s 33ms/step - loss: 0.0413 - accuracy: 0.9908
Epoch 10/50
105/105 [==============================] - 4s 33ms/step - loss: 0.0177 - accura

In [76]:
## SAving the baseline stage 1 DNN
model.save('Stage1_DNN.h5')